# Getting started

The package we will use for our network analysis is `igraph`. There are also other packages out there for Python, such as `networkx` or `graphtool`, but will not use them during this course.

In [ ]:
import igraph as ig

Note that at any time, you can get some information by pressing Shift-Tab when your cursor is on some function.

We can create a graph ourselves, adding vertices and edges as we go.

In [ ]:
G = ig.Graph(directed=False)
G.add_vertices(n=4)
G.add_edges([(0, 1),
             (0, 2),
             (0, 3),
             (1, 2),
             (1, 3),
             (2, 3)])

We can get a summary of the graph, providing some basic information on the graph

In [ ]:
G.summary()

The result indicates this graph is undirected (indicated by the `U`) has 4 nodes and 6 edges. The information on the number of nodes and edges can also be obtained using functions.

In [ ]:
n = G.vcount()
m = G.ecount()
print('{0} nodes, {1} edges'.format(n, m))

In the remainder of this exercise, we will work with a built-in graph that is quite famous: the karate club network constructed by [Zachary (1977)](https://www.jstor.org/stable/3629752?seq=1#page_scan_tab_contents).

In [ ]:
G = ig.Graph.Famous('Zachary')

You can plot the graph easily

In [ ]:
G['layout'] = G.layout_fruchterman_reingold()
G.vs['color'] = 'gray'
G.es['color'] = 'gray'
ig.plot(G)

The most basic elements of any graph are the *nodes* and the connection between the nodes, called *edges*. Nodes are also called *vertices* and edges are also called *links*. These words will be used interchangebly. Vertices and edges are the terms that are most often used in graph theory, while nodes and links are more common in (social) network analysis. Sometimes, you also see the term *arc* for referring to an edge.

In `igraph` the terms vertex and edges are used throughout, and they can be accessed through a so-called `VertexSequence` and `EdgeSequence`. You can make different selections of vertices and edges, either based on attributes or simply specifying specific (vertex or edge) indices. The functionality on vertex and edge sequences is quite extensive. The following just demonstrates one possibility.

In [ ]:
vs = G.vs[[0, 1, 2, 3]]
es = G.es.select(_within=vs)
vs['color'] = 'blue'
es['color'] = 'blue'
ig.plot(G)

The nodes that are linked to another node are called the *neighbors* of a node. The number of neighbours is called the *degree* of a node.

In [ ]:
neighbors = G.neighbors(4)
print(neighbors, G.degree(4))

# Paths

One of the most basic operations on any network is finding a *path* from one node to another node, not unlike finding a route from one city to another using some navigational software.

In [ ]:
edge_paths = G.get_shortest_paths(v=16, to=15, output='epath')
edge_paths

This retrieves a shortest path between node `16` and node `15`. It returns the *indices* of the edges because we set `output='epath'`. In this case there is only one path of 5 edges long. We can also get the endpoints of those edges, so that the path becomes a bit more clear.

In [ ]:
edge_path = G.es[edge_paths[0]];
[(edge.source, edge.target) for edge in edge_path]

We can also get the same path in terms of vertices

In [ ]:
vertex_path = G.vs[G.get_shortest_paths(v=16, to=15, output='vpath')[0]]
[v.index for v in vertex_path]

We can visualize this path as follows.

In [ ]:
G.vs['color'] = 'gray'
vertex_path['color'] = 'red'

G.es['color'] = 'gray'
G.es['width'] = 0.5
edge_path['color'] = 'red'
edge_path['width'] = 2

ig.plot(G)

Rather than determining the actual shortest paths, we may also simply be interested in the distance between two nodes.

In [ ]:
G.shortest_paths(source=16, target=15)

This only provides the actual length of the path (5 edges) rather than the actual path. We can also use this function to get the distance of all nodes to all other nodes. This is conveniently represented as a matrix, for which the `numpy` library is especially well suited.

In [ ]:
import numpy as np
distances = np.array(G.shortest_paths())
distances

The largest distance from a node to any other node is called the *eccentricity*. If a node has a very high eccentricity, it is a bit peripheral. If a node has a very low eccentricity, it means that it is relatively close to to all other nodes, and that it is in the center of the graph in a certain sense.

In [ ]:
eccentricity = distances.max(1)
print(eccentricity)
ig.plot(G, vertex_size=5*(6 - eccentricity))

The minimum eccentricity is called the *radius* of the graph and the maximum eccentricity is called the *diameter*. The diameter of any graph is always larger than the radius and smaller than twice the radius.

The network we currently look at is connected, which is not necessarily the case. It can consist of multiple components. Let us delete a number of edges so that we get a network with two components.

In [ ]:
G.delete_edges(G.es.select(_between=[(4, 5, 6, 10), (0,)]))
ig.plot(G)

The path we constructed earlier is now no longer connected. In this case, there is no longer any path between the two nodes at all. The distance between the two nodes in hence infinitely large.

In [ ]:
G.shortest_paths(source=16, target=15)

There is now no longer a path between any two nodes, and the graph is then no longer said to be *connected*.

In [ ]:
G.is_connected()

The network is now said to be *disconnected* and the different parts of the network that still are connected are called *connected components*.

In [ ]:
components = G.clusters()
G.vs[components[0]]['color'] = 'red'
G.vs[components[1]]['color'] = 'blue'
ig.plot(G)

Usually, networks tend to have one large component, and many smaller components. In empirical networks, it is therefore common practice to restrict any further analyses to the largest connected component. Because the difference between the largest connected component and the other components is usually quite large, the largest connected component is sometimes also called the *giant component*.

In [ ]:
H = G.clusters().giant()
ig.plot(H, layout=H.layout_fruchterman_reingold())

# Cycles

If there are two paths going from one node to another node, we can join them together and make a *cycle*. We start again with a fresh graph.

In [ ]:
G = ig.Graph.Famous('Zachary')
G['layout'] = G.layout_fruchterman_reingold()

In [ ]:
cycle = G.es[G.get_eids(path=(24, 25, 23, 27, 24))]

G.es['color'] = 'gray'
G.es['width'] = 0.5
cycle['color'] = 'red'
cycle['width'] = 2

ig.plot(G)

This is a cycle of length 4, but longer cycles do exist in the graph. The smallest cycle of length 3 are of particular interest to the social sciences. A small group of three people is commonly called a *triad* in the social sciences. It is of particular interest because if two people have a friend in common, they are more likely to become friends themselves. This is known as *triadic closure*.

If there are many closed triads, i.e. many cycles of length 3, it suggests that many people have many friends in common. In particular, the network is then said to be *clustered*. The *clustering coefficient* for a node is defined as the proportion of neighbours that are connected amongst each other. A clustering coefficient of 1 then means that all the neighbours of a node are connected to each other, while a clustering coefficient of 0 means that no neigbhours are connected amongst each other. The overall clustering is then simply the average over the whole network.

In [ ]:
clustering = G.transitivity_local_undirected(mode="zero")

ig.plot(G, vertex_size=10*(np.array(clustering)+0.5))

As you can see, the nodes that are more clustered seem to be more peripheral. This is something that can be seen more often. Nodes that have many neighbours usually have fewer connections between *all* their neighbours, so that they tend to have a lower clustering coefficient.

This may also suggest that some nodes function as bridges, connecting different parts of the network. For example, when removing node 0 from the network, this disconnects the network.

In [ ]:
H = G.copy()
H.delete_vertices(0)
components = H.clusters()
ig.plot(components, layout=H.layout_fruchterman_reingold())

This means that node 0 functions as a bridge. In fact, this is the only strict bridge in this graph.

In [ ]:
is_bridge = [False]*G.vcount()
for v in range(G.vcount()):
    H = G.copy()
    H.delete_vertices(v)
    is_bridge[v] = len(H.clusters()) > 1
    
ig.plot(G, vertex_color=is_bridge, palette=ig.RainbowPalette(2))

# Social network analysis

## Homophily

If two nodes are more likely to be connected when they share a certain attribute, this is known as *homophily*. We will illustrate this on a network of contact between pupils from a high school available from [SocioPatterns](http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/). The contacts were collected through devices that would automatically record face-to-face interaction (i.e. physical proximity) between pupils during 5 days.

In [ ]:
import ighelper
G = ighelper.GraphFromURL('https://storage.googleapis.com/css-files/sociopatterns.graphml')
G = G.clusters().giant()

G.es['width'] = 0.01*np.array(G.es['weight'])
G['layout'] = G.layout_fruchterman_reingold(weights='weight')
G.vs['color'] = ['pink' if v['gender'] == 'F' else 'blue' for v in G.vs]

ig.plot(G)

In the above plot, the male pupils are colored blue, and the females pink. Now the central question is whether males and femals tend to have more links to the same sex, or more evenly distributed. The more general term for homophily is *assortativity*, which refers to how likely two nodes that have the same attribute are linked. In the plot above it is not immediately clear whether there is gender assortativity (or homophily) in this network. But using a statistical measure of assortativity gives us more insight.

In [ ]:
G.assortativity_nominal(ig.VertexClustering.FromAttribute(G, 'gender').membership)

Like a correlation, the assortativity would be 0 if gender would have no relation with the connectivity, 1 if all connections would only be between nodes of the same class and -1 if all connections would only be between nodes of different classes. An assortativity of 0.17 is then not very high, but it does indicate some tendency for pupils of the same sex to be linked.

The pupils mostly associate with other people from the same (school)class. This is again clear from the assortativity based on the (school)class.

In [ ]:
G.assortativity_nominal(ig.VertexClustering.FromAttribute(G, 'class').membership)

If we look at the gender assortativity per class this drops to 0. This suggests that the overall assortativity of 0.17 due to gender imbalance in the classes.

In [ ]:
classes = ig.VertexClustering.FromAttribute(G, 'class');
for H in classes.subgraphs():
    gender_ratio = sum([v['gender'] == 'F' for v in H.vs])/float(H.vcount())
    gender_assortativity = H.assortativity_nominal(ig.VertexClustering.FromAttribute(H, 'gender').membership)
    print('class: {0},\t% female: {1:.2f},\tassortativity: {2: .3f}'.format(H.vs['class'][0], gender_ratio, gender_assortativity))

## Social influence

One of the key concepts in social network analysis is social influence. The basic idea is that people that are linked influence each other. For example, if person A holds opinion A but all his neighbors hold opinion B, then he is likely to switch to opinion A. Let us simulate some simple opinion dynamics on the highschool network. We will assume that every node will switch to the majority opinion in its neighborhood, and we will start from some random initial condition.

In [ ]:
G = ighelper.GraphFromURL('https://storage.googleapis.com/css-files/sociopatterns.graphml')
G = G.clusters().giant()

In [ ]:
G.vs['opinion'] = 0
G.vs['new_opinion'] = np.random.randint(2, size=G.vcount())

while G.vs['opinion'] != G.vs['new_opinion']:
    G.vs['opinion'] = G.vs['new_opinion']
    for v in G.vs:
        v['new_opinion'] = 1*(sum(G.vs[G.neighbors(v)]['opinion']) > 0.5*v.degree())

#G.vs[v.neighbors()]['opinion']
ig.plot(G, vertex_color=G.vs['opinion'], palette=ig.RainbowPalette(2))

One of the problems with social influence is that it usually is confounded with homophily. Let us check if there is assortativity on the result of the opinion dynamics.

In [ ]:
G.assortativity_nominal(G.vs['opinion'])

The assortativity on the opinion is roughly the same as the assortativity on the classes. But then begs the question of whether homophily was operating or social influence. The cases here are clear. The class likely influences the probability of a link, so homophily is operating. In the opinion dynamics simulation, we know that the network did not change, but only the opinion did, so there is clear social influence. But if we empirically see that there is some assortativity on some variable, we cannot determine unambiguously whether there is homophily or social influence.

## Centrality

## Weak ties

## Structural holes

## Equivalence

## Social balance

# Complex networks

## Random graphs

## Spreading & percolation

## Community detection